In [ ]:
%run oeai_py

In [ ]:
# Create an instance of OEAI class and set the platform ("Synapse" or "Fabric")
oeai = OEAI("Fabric")

In [ ]:
# CHANGE VALUES FOR YOUR KEY VAULT
keyvault = "INSERT_YOUR_KEYVAULT_NAME" # fully qualified for Fabric 
keyvault_linked_service = "INSERT_YOUR_KEYVAULT_LINKED_SERVICE_NAME" # linked service name for Synapse

# Synapse OEA environment path & secrets
bronze_path = oeai.get_secret(spark, "renaissance-bronze", keyvault_linked_service, keyvault)
silver_path = oeai.get_secret(spark, "renaissance-silver", keyvault_linked_service, keyvault)
school_ids_secret = oeai.get_secret(spark, "renaissance-school-ids", keyvault_linked_service, keyvault)
subdirectories = school_ids_secret.split(",")

In [ ]:
school_ids_secret = oeai.get_secret(spark, "renaissance-school-ids", keyvault_linked_service, keyvault)
subdirectories = school_ids_secret.split(",")
# print(subdirectories)

In [ ]:
from pyspark.sql import SparkSession
import zipfile
from io import BytesIO
import pandas as pd
from urllib.parse import quote

base_path = bronze_path

# Function to extract AR_UK.csv from a zip file
def extract_csv_from_zip(zip_data):
    with zipfile.ZipFile(BytesIO(zip_data), 'r') as zip_ref:
        for file in zip_ref.namelist():
            if file.endswith("AR_UK.csv"):
                with zip_ref.open(file) as f:
                    return pd.read_csv(f)

# List to hold DataFrames
df_list = []

# Loop through zip files in ADLS, extract AR_UK.csv, and combine them into a DataFrame
for subdir in subdirectories:
    zip_file_path = f"{base_path}{subdir}"
    print(f"Checking file: {zip_file_path}")  # Debug statement
    try:
        binary_files_df = spark.read.format("binaryFile").load(zip_file_path)
        zip_data = binary_files_df.select("content").collect()[0][0]
        pdf = extract_csv_from_zip(zip_data)
        if pdf is not None:
            df = spark.createDataFrame(pdf)
            df_list.append(df)
    except Exception as e:
        print(f"Failed to process {zip_file_path}: {e}")

# Combine all DataFrames if there are any
if df_list:
    combined_df = df_list[0]
    for df in df_list[1:]:
        combined_df = combined_df.union(df)
    # Show the combined DataFrame
    combined_df.show()
    # Save the combined DataFrame as a Delta table
    delta_path = silver_path + "fact_AcceleratedReader"
    combined_df.write.format("delta").mode("overwrite").save(delta_path)
else:
    print("No files were processed.")
